In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
from models.models import V_CNN
from models.models import VAE

In [3]:
# Set random seed for reproducibility
torch.manual_seed(42)

In [4]:
# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

# Define data loaders
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


# Create an instance of the VAE
vae = VAE().to(device)

# Define the loss function and optimizer for VAE
vae_criterion = nn.BCELoss()
vae_optimizer = optim.Adam(vae.parameters(), lr=0.001)

# Training loop for VAE
vae_num_epochs = 10

def vae_loss_function(recon_x, x, mu, logvar):
    # Reconstruction loss
    recon_loss = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # KL divergence loss
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Total VAE loss
    vae_loss = recon_loss + kl_loss

    return vae_loss


for epoch in range(vae_num_epochs):
    vae.train()
    train_loss = 0.0

    for batch_idx, (images, _) in enumerate(train_loader):
        images = images.to(device)

        vae_optimizer.zero_grad()
        recon_images, mu, logvar = vae(images)
        #vae_loss = vae_loss_function(recon_images, images, mu, logvar)
        vae_loss = vae_loss_function(recon_images.view(-1, 784), images.view(-1, 784), mu, logvar)
        vae_loss.backward()
        vae_optimizer.step()

        train_loss += vae_loss.item()

        if (batch_idx + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{vae_num_epochs}], Step [{batch_idx + 1}/{len(train_loader)}], '
                  f'VAE Loss: {vae_loss.item():.4f}')

    print(f'Train VAE Loss: {train_loss / len(train_loader):.4f}')

Epoch [1/10], Step [100/938], VAE Loss: -965839.3125
Epoch [1/10], Step [200/938], VAE Loss: -1216184.1250
Epoch [1/10], Step [300/938], VAE Loss: -1467345.5000
Epoch [1/10], Step [400/938], VAE Loss: -1764519.8750
Epoch [1/10], Step [500/938], VAE Loss: -1941797.3750
Epoch [1/10], Step [600/938], VAE Loss: -2009008.2500
Epoch [1/10], Step [700/938], VAE Loss: -2021194.1250
Epoch [1/10], Step [800/938], VAE Loss: -2079265.7500
Epoch [1/10], Step [900/938], VAE Loss: -2120486.7500
Train VAE Loss: -1621779.2911
Epoch [2/10], Step [100/938], VAE Loss: -2132811.0000
Epoch [2/10], Step [200/938], VAE Loss: -2161532.5000
Epoch [2/10], Step [300/938], VAE Loss: -2299018.2500
Epoch [2/10], Step [400/938], VAE Loss: -2187028.2500
Epoch [2/10], Step [500/938], VAE Loss: -2325315.7500
Epoch [2/10], Step [600/938], VAE Loss: -2249390.7500
Epoch [2/10], Step [700/938], VAE Loss: -2290302.0000
Epoch [2/10], Step [800/938], VAE Loss: -2287128.7500
Epoch [2/10], Step [900/938], VAE Loss: -2347751.7500

In [5]:
train_features = []
train_labels = []

with torch.no_grad():
    for images, labels in train_loader:
        images = images.to(device)
        mu, logvar = vae.encode(images)
        features = vae.reparameterize(mu, logvar)
        train_labels.extend(labels.numpy())
        train_features.append(features.unsqueeze(1))

train_features = torch.cat(train_features, dim=0)
train_features = train_features.view(60000,1,8,8)

# Print the shape of train_features
print(train_features.shape)

torch.Size([60000, 1, 8, 8])


In [6]:
test_features = []
test_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        mu, logvar = vae.encode(images)
        features = vae.reparameterize(mu, logvar)
        test_labels.extend(labels.numpy())
        test_features.append(features.unsqueeze(1))

test_features = torch.cat(test_features, dim=0)
test_features = test_features.view(10000,1,8,8)

# Print the shape of test_features
print(test_features.shape)

torch.Size([10000, 1, 8, 8])


In [7]:
# Convert the features to tensors
train_features = torch.tensor(train_features)
train_labels = torch.tensor(train_labels)
test_features = torch.tensor(test_features)
test_labels = torch.tensor(test_labels)

# Create new datasets with the extracted VAE features
train_dataset = torch.utils.data.TensorDataset(train_features, train_labels)
test_dataset = torch.utils.data.TensorDataset(test_features, test_labels)

# Define data loaders for the new datasets
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

/tmp/ipykernel_77629/1021885846.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_features = torch.tensor(train_features)
/tmp/ipykernel_77629/1021885846.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_features = torch.tensor(test_features)


In [8]:
# Create an instance of the CNN
model = V_CNN().to(device)

# Define the loss function and optimizer for the CNN
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop for the CNN
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (features, labels) in enumerate(train_loader):
        features, labels = features.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if (batch_idx + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(train_loader)}], '
                  f'Loss: {loss.item():.4f}')

    print(f'Train Accuracy: {(100 * correct / total):.2f}%')

# Evaluation on the test set
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate test accuracy
test_accuracy = (100 * correct / total)
test_loss /= len(test_loader)
print(f'Test Accuracy: {test_accuracy:.2f}%')
print(f'Test Loss: {test_loss:.4f}')

Epoch [1/10], Step [100/938], Loss: 0.6387
Epoch [1/10], Step [200/938], Loss: 0.3592
Epoch [1/10], Step [300/938], Loss: 0.2782
Epoch [1/10], Step [400/938], Loss: 0.4082
Epoch [1/10], Step [500/938], Loss: 0.2477
Epoch [1/10], Step [600/938], Loss: 0.2072
Epoch [1/10], Step [700/938], Loss: 0.1504
Epoch [1/10], Step [800/938], Loss: 0.1352
Epoch [1/10], Step [900/938], Loss: 0.1787
Train Accuracy: 88.30%
Epoch [2/10], Step [100/938], Loss: 0.1034
Epoch [2/10], Step [200/938], Loss: 0.1634
Epoch [2/10], Step [300/938], Loss: 0.2081
Epoch [2/10], Step [400/938], Loss: 0.1668
Epoch [2/10], Step [500/938], Loss: 0.1700
Epoch [2/10], Step [600/938], Loss: 0.0698
Epoch [2/10], Step [700/938], Loss: 0.0758
Epoch [2/10], Step [800/938], Loss: 0.1939
Epoch [2/10], Step [900/938], Loss: 0.0725
Train Accuracy: 94.64%
Epoch [3/10], Step [100/938], Loss: 0.1599
Epoch [3/10], Step [200/938], Loss: 0.2285
Epoch [3/10], Step [300/938], Loss: 0.0777
Epoch [3/10], Step [400/938], Loss: 0.2277
Epoch [3